### Découpage par lesions 

In [1]:
import os
import csv
import SimpleITK as sitk
import numpy as np

In [4]:
import os
import csv
import SimpleITK as sitk

# Dossier racine du dataset (un niveau au-dessus de 'train')
ROOT_DIR = "../../../../27919209/MSLesSegDataset"
# Dossiers à parcourir
SPLITS = ["train", "test"]

def classify(lesion_mean, ref_mean, ref_std, factor=1.5):
    """
    Classification basée sur le Z-score.
    """
    upper_bound = ref_mean + (factor * ref_std)
    lower_bound = ref_mean - (factor * ref_std)
    
    if lesion_mean > upper_bound:
        return "hypersignal"
    elif lesion_mean < lower_bound:
        return "hyposignal"
    else:
        return "isosignal"

output_csv = "lesion_per_lesion_signal_stats_v2.csv"

with open(output_csv, mode="w", newline="") as f:
    writer = csv.writer(f)
    # Ajout de la colonne 'split' pour la traçabilité
    writer.writerow([
        "patient",
        "session",
        "split",
        "lesion_id",
        "modality",
        "lesion_mean",
        "lesion_min",
        "lesion_max",
        "ref_mean",
        "ref_std",
        "signal_class"
    ])

    # --- Boucle sur les dossiers (train puis test) ---
    for split in SPLITS:
        split_dir = os.path.join(ROOT_DIR, split)
        
        if not os.path.isdir(split_dir):
            print(f"[ERROR] Dossier {split_dir} non trouvé.")
            continue

        print(f"--- Début du traitement du dossier : {split} ---")

        # --- Boucle sur les patients ---
        for patient in sorted(os.listdir(split_dir)):
            patient_dir = os.path.join(split_dir, patient)
            if not os.path.isdir(patient_dir):
                continue

            # --- Boucle sur les sessions (T1, T2, etc.) ---
            for session in sorted(os.listdir(patient_dir)):
                session_dir = os.path.join(patient_dir, session)
                if not os.path.isdir(session_dir):
                    continue

                prefix = f"{patient}_{session}"

                # Chemins des fichiers
                mask_path  = os.path.join(session_dir, f"{prefix}_MASK.nii.gz")
                t1_path    = os.path.join(session_dir, f"{prefix}_T1.nii.gz")
                t2_path    = os.path.join(session_dir, f"{prefix}_T2.nii.gz")
                flair_path = os.path.join(session_dir, f"{prefix}_FLAIR.nii.gz")

                # Vérification de l'existence de tous les fichiers nécessaires
                if not (os.path.exists(mask_path)
                        and os.path.exists(t1_path)
                        and os.path.exists(t2_path)
                        and os.path.exists(flair_path)):
                    print(f"[WARN] Fichiers manquants pour {split}/{patient}/{session}, on saute.")
                    continue

                print(f"Traitement {split} : {patient} / {session} ...")

                # Lecture des images
                mask_img_orig  = sitk.ReadImage(mask_path)
                t1_img_orig    = sitk.ReadImage(t1_path)
                t2_img_orig    = sitk.ReadImage(t2_path)
                flair_img_orig = sitk.ReadImage(flair_path)

                # Traitement
                lesion_bin = mask_img_orig > 0
                lesion_cc = sitk.ConnectedComponent(lesion_bin)
                brain_bin = t1_img_orig > 0
                ref_bin   = sitk.And(brain_bin, sitk.Not(lesion_bin))
                ref_label = sitk.Cast(ref_bin, sitk.sitkUInt8)

                modalities = {
                    "T1": sitk.Cast(t1_img_orig, sitk.sitkFloat64),
                    "T2": sitk.Cast(t2_img_orig, sitk.sitkFloat64),
                    "FLAIR": sitk.Cast(flair_img_orig, sitk.sitkFloat64)
                }

                for modality_name, img in modalities.items():
                    stats_ref = sitk.LabelStatisticsImageFilter()
                    stats_ref.Execute(img, ref_label)
                    
                    if not stats_ref.HasLabel(1):
                        continue
                    
                    ref_mean = stats_ref.GetMean(1)
                    ref_std  = stats_ref.GetSigma(1)

                    stats_lesions = sitk.LabelStatisticsImageFilter()
                    stats_lesions.Execute(img, lesion_cc)

                    for lesion_id in stats_lesions.GetLabels():
                        if lesion_id == 0:
                            continue

                        lesion_mean = stats_lesions.GetMean(lesion_id)
                        lesion_min  = stats_lesions.GetMinimum(lesion_id)
                        lesion_max  = stats_lesions.GetMaximum(lesion_id)

                        signal_class = classify(lesion_mean, ref_mean, ref_std)

                        writer.writerow([
                            patient,
                            session,
                            split, # On écrit si c'est 'train' ou 'test'
                            int(lesion_id),
                            modality_name,
                            round(float(lesion_mean), 4),
                            round(float(lesion_min), 4),
                            round(float(lesion_max), 4),
                            round(float(ref_mean), 4),
                            round(float(ref_std), 4),
                            signal_class
                        ])

print(f"Terminé. Résultats regroupés dans {output_csv}")

--- Début du traitement du dossier : train ---
Traitement train : P1 / T1 ...
Traitement train : P1 / T2 ...
Traitement train : P1 / T3 ...
Traitement train : P10 / T1 ...
Traitement train : P10 / T2 ...
Traitement train : P11 / T1 ...
Traitement train : P11 / T2 ...
Traitement train : P12 / T1 ...
Traitement train : P12 / T2 ...
Traitement train : P12 / T3 ...
Traitement train : P12 / T4 ...
Traitement train : P13 / T1 ...
Traitement train : P13 / T2 ...
Traitement train : P14 / T1 ...
Traitement train : P14 / T2 ...
Traitement train : P14 / T3 ...
Traitement train : P14 / T4 ...
Traitement train : P15 / T1 ...
Traitement train : P16 / T1 ...
Traitement train : P17 / T1 ...
Traitement train : P18 / T1 ...
Traitement train : P19 / T1 ...
Traitement train : P19 / T2 ...
Traitement train : P19 / T3 ...
Traitement train : P19 / T4 ...
Traitement train : P2 / T1 ...
Traitement train : P2 / T2 ...
Traitement train : P2 / T3 ...
Traitement train : P2 / T4 ...
Traitement train : P20 / T1 ...
